In [1]:
# Current directory
import os
os.chdir('F:/One/ACP')

### Stacking model

In [2]:
import numpy as np
import pandas as pd


# train
train_features = pd.read_csv("features_label/train.csv", index_col=False, header=None)
train_labels = pd.read_csv("features_label/train_label.csv", index_col=False, header=None)
# test
test_features = pd.read_csv("features_label/test.csv", index_col=False, header=None)
test_labels = pd.read_csv("features_label/test_label.csv", index_col=False, header=None)

print('train_features:', train_features.shape)
print('train_labels', train_labels.shape)
print('test_features', test_features.shape)
print('test_labels', test_labels.shape)


train_features = np.array(train_features)
train_labels = np.array(train_labels)
test_features = np.array(test_features)
test_labels = np.array(test_labels)


train_features: (1378, 320)
train_labels (1378, 1)
test_features (344, 320)
test_labels (344, 1)


In [3]:
import math
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import adam_v2
from keras.layers import Dense,Dropout
from sklearn.preprocessing import scale
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, Dense,AveragePooling1D
from sklearn.metrics import confusion_matrix, roc_curve, auc


# model
def CNN(input_dim, out_dim, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters = 64, kernel_size = 3, padding = 'same', activation= 'relu'))
    model.add(AveragePooling1D(pool_size=2,strides=1,padding="SAME"))
    model.add(Conv1D(filters = 64, kernel_size =  3, padding = 'same', activation= 'relu'))
    model.add(AveragePooling1D(pool_size=2,strides=1,padding="SAME"))
    model.add(Flatten())
    model.add(Dense(int(32), activation = 'relu'))
    model.add(Dense(out_dim, activation = 'softmax',name="Dense_2"))
    model.compile(loss = 'binary_crossentropy', optimizer = adam_v2.Adam(learning_rate), metrics =['binary_accuracy'])
    return model


def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)


def to_categorical(y, nb_classes=None):
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1
    return Y


In [45]:
# 定义要调节的参数
# LR
# param_grid = {
#     'penalty': ['l2', 'l1'],
#     'C': [0.01, 0.1, 1, 10, 100],
#     'solver': ['liblinear', 'saga']
# }
# RF
# param_grid = {
#     'n_estimators': [50, 100, 200],   # 森林中树的数量
#     'max_depth': [None, 10, 20, 30],  # 每棵树的最大深度
#     'min_samples_split': [2, 5, 10],  # 分裂节点所需的最小样本数
#     'min_samples_leaf': [1, 2, 4],    # 叶子节点的最小样本数
#     'bootstrap': [True, False]        # 是否使用 bootstrap 样本
# }
# XGBoost
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'subsample': [0.6, 0.8, 1.0]
# }
# SVM
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'kernel': ['linear', 'rbf', 'poly'],
#     'gamma': ['scale', 'auto']
# }
# KNN
param_grid = {
    'n_neighbors': [3, 5, 7, 10],  # 邻居数量
    'weights': ['uniform', 'distance'],  # 权重：统一权重或根据距离加权
    'p': [1, 2]  # 距离度量：p=1 曼哈顿距离，p=2 欧氏距离
}



In [4]:
import math
import numpy as np
import xgboost as xgb
from sklearn.svm import SVC  # 导入 SVM 模型
from tensorflow.keras.models import load_model
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  # 导入 KNN 模型
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  # 导入随机森林
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc


# Create a directory to save the model
model_GRU_dir = 'save_models/GRU/Independence'
model_CNN_dir = 'save_models/CNN/Independence'
model_CapsuleGAN_dir = 'save_models/CapsuleGAN/Independence'

[sample_num, input_dim] = np.shape(train_features)
y_train = train_labels
y_test = test_labels
out_dim=2

ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]

# Prepare for stacking model
stacked_features = []
stacked_labels = []
stacked_features_train = []
stacked_labels_train = []

for i in range(10):
    # Loading model
    if i in (0, 1):
        X_train = np.reshape(train_features, (-1,1,input_dim))
        X_test = np.reshape(test_features, (-1,1,input_dim))

        model_path = os.path.join(model_GRU_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')
    elif i in (2, 3, 4, 5, 6, 7):
        X_train = np.reshape(train_features, (-1,1,input_dim))
        X_test = np.reshape(test_features, (-1,1,input_dim))

        model_path = os.path.join(model_CNN_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')
    else:
        X_train = train_features
        X_test = test_features

        model_path = os.path.join(model_CapsuleGAN_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')

    y_score_train = clf.predict(X_train)
    y_score = clf.predict(X_test)

    # Collect features for stacking
    if len(stacked_features) == 0:
        stacked_features_train = y_score_train
        stacked_features = y_score
        stacked_labels_train = y_train
        stacked_labels = y_test
    else:
        stacked_features_train = np.hstack([stacked_features_train, y_score_train])
        stacked_features = np.hstack([stacked_features, y_score])


[sample_num, input_dim] = np.shape(stacked_features_train)
stacked_features_train = np.reshape(stacked_features_train, (-1,1,input_dim))
stacked_features = np.reshape(stacked_features, (-1,1,input_dim))
out_dim=2
labels_train = to_categorical(y_train)

# 在fit函数之前，将y_ind_test转换为一维数组
# stacked_labels_train = stacked_labels_train.ravel()
# stacked_labels = stacked_labels.ravel()

# Train meta-learner (stacking classifier) with the collected features
# meta_learner = LogisticRegression(max_iter=5000)        # 逻辑回归
# meta_learner = RandomForestClassifier()     # 随机森林
# meta_learner = xgb.XGBClassifier(eval_metric='logloss')   # XGBoost
# meta_learner = SVC(probability=True)
# meta_learner = KNeighborsClassifier()
# meta_learner = GridSearchCV(meta_learner, param_grid, cv=5, scoring='accuracy')     # 最佳参数搜索
# meta_learner.fit(stacked_features_train, stacked_labels_train)
# 输出最优参数
# print("Best parameters: ", meta_learner.best_params_)

meta_learner = CNN(input_dim, out_dim, 0.0001)      # CNN
meta_learner.fit(stacked_features_train, labels_train, batch_size=64, epochs=30)


# Get final predictions from meta-learner
# DL
pred_score = meta_learner.predict(stacked_features)
# ML
# pred_score = meta_learner.predict_proba(stacked_features)
final_predictions = categorical_probas_to_classes(pred_score)

TP, FP, FN, TN = confusion_matrix(y_test, final_predictions).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
Sn_collecton = TP/(TP+FN)
Sp_collecton = TN/(TN+FP)
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton = MCC
BACC_collecton = 0.5*TP/(TP+FN)+0.5*TN/(TN+FP)
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton = ACC
# ROC curve
fpr, tpr, _ = roc_curve(y_test, pred_score[:, 1])
auc_roc = auc(fpr, tpr)
AUC_collecton = auc_roc
# PR curve
precision, recall, _ = precision_recall_curve(y_test, pred_score[:, 1])
average_precision = average_precision_score(y_test, pred_score[:, 1])
AP = average_precision

# Output
results = [
    f"ACC: {round(ACC_collecton, 3)}",
    f"BACC: {round(BACC_collecton, 3)}",
    f"Sn: {round(Sn_collecton, 3)}",
    f"Sp: {round(Sp_collecton, 3)}",
    f"MCC: {round(MCC_collecton, 3)}",
    f"AUC: {round(AUC_collecton, 3)}",
    f"AP: {round(AP, 3)}"
]

for result in results:
    print(result)

# Append the results to the file
with open('result/results_Stack.txt', 'a') as file:
    file.write("----------------------------------------\n")
    for result in results:
        file.write(result + '\n')


Model for fold 0 loaded from save_models/GRU/Independence\ESM_0.h5
Model for fold 1 loaded from save_models/GRU/Independence\ESM_1.h5
Model for fold 2 loaded from save_models/CNN/Independence\ESM_2.h5
Model for fold 3 loaded from save_models/CNN/Independence\ESM_3.h5
Model for fold 4 loaded from save_models/CNN/Independence\ESM_4.h5
Model for fold 5 loaded from save_models/CNN/Independence\ESM_5.h5
Model for fold 6 loaded from save_models/CNN/Independence\ESM_6.h5
Model for fold 7 loaded from save_models/CNN/Independence\ESM_7.h5
Model for fold 8 loaded from save_models/CapsuleGAN/Independence\ESM_8.h5
Model for fold 9 loaded from save_models/CapsuleGAN/Independence\ESM_9.h5
Epoch 1/30
22/22 [==============================] - 4s 15ms/step - loss: 0.6690 - binary_accuracy: 0.6988
Epoch 2/30
22/22 [==============================] - 0s 5ms/step - loss: 0.6435 - binary_accuracy: 0.8650
Epoch 3/30
22/22 [==============================] - 0s 5ms/step - loss: 0.6157 - binary_accuracy: 0.8716


In [5]:
# 保存ROC曲线相关参数
np.savez('graph/ROC/Stack.npz', fpr=fpr, tpr=tpr, roc_auc=AUC_collecton)

# 保存PR曲线相关参数
np.savez('graph/PR/Stack.npz', recall=recall, precision=precision, average_precision=AP)